# Lab 4: Búsquedas facetadas

En este notebook vamos a explorar las diferentes capacidades de búsqueda que nos ofrece Elasticserach. En concreto vamos a ver como construir un buscador por facetas como el que vemos en la siguiente imagen.

<img src="../../images/els/els31.jpg" alt="Busqueda autocompletada"/>

Para realizar los ejercicios propuestos en este notebook, puedes utilizar el Dev Tools de Kibana. Puedes acceder a él a través de este enlace: http://127.0.0.1:5601

Para poder construir este tipo de buscadores vamos a utilizar el framework de agregación de Elasticsearch y más en concreto las **Bucket Aggregations** que nos van a permitir organizar grandes datasets en gurpos dependiendo del valor de un campo.

# Búsquedas facetadas

Cuando implementamos un buscador con facetas, entendemos por faceta cada una de las propiedades que tienen nuestros resgistros por las que son susceptibles de ser filatradas en una búsqueda. En el ejemplo de la imagen, las facetas son cada una de las propiedades de nuestro producto.
Para implementar el buscador, lo que nos interesa es poder mostrar los diferentes valores de cada una de las propiedades, de tal forma que el usuario pueda seleccionar de forma sencilla los valores de cada propiedad por los que filtrar los resultados.

En el caso concreto de Elasticsearch, podemos ver las facetas como cada uno de los diferentes campos de nuestro documento. Por lo que a la hora de poponer las diferentes altenativas para los valares de cada una de las facetas, necesitamos poder realizar una consulta que nos devuelva los diferentes valores para los campos que representan nuestras facetas.

Para realizar esta consuta necesitamos poder realizar busquedas agregadas. Elasticsearch nos proporciona un framework de agregación con dos tipos principales de agregaciones:

* **Bucket aggregations**: nos van a permitir organizar grandes datasets en gurpos dependiendo del valor de un campo.
* **Metric aggregations**: nos permiten calacular métricas como count, sum, min, max, avg... sobre campos de tipo numérico.

Para construir los filtros facetados se utilizan as agregaciones de tipo Bucket Aggregation. 

## Bucket Aggregations

Las bucket aggregarions no generan métricas sobre los campos, sino que generan gurpos o buckets de documentos asociados a un critério que indica si un documento entra o no en ese bucket. Además nos indica cuantos documentos han entrado en cada uno de los buckets.

El número máximo de buckets que puede devolver una consulta está limitado a nivel de cluster por el parámetro de configuración search.max_buckets, por defecto 65.536.

Las agregaciones de tipo bucket generan como resultado buckets, donde cada bucket está asociado con una clave y a un criterio de selección. 

Cuando se ejecuta la agregación, todos los criterios asociados a cada bucket se evalúan sobre cada documento que encontramos en el contexto de búsqueda y si estos criterios se cumplen se incluye en el bucket. 

Al final de la ejecución se obtiene una lista de los buckets con el conjunto de documentos que pertenece a cada uno de ellos.

Dependiendo del tipo del campo sobre el que vamos ha hacer las facetas utilizaremos las siguientes agregaciones (en este notebook sólo vamos a ver las más comunes. Para ver el total de agregaciones que se pueden realizar en Elasticsearch consultar la siguiente página de la documentación: https://www.elastic.co/guide/en/elasticsearch/reference/8.4/search-aggregations-bucket.html):

* Texto:
    * **Terms Aggregation**: los buckets se generan de forma dinámica para cada uno de los valores únicos que se encuentran en los campos especificados en la agregación.
* Números:
    * **Range Aggregation**
    * **Histogram Aggregation**
* Fechas:
    * **Date Range Aggregation**
    * **Date Histogram Aggregation**


## 1. Terms Aggregation

Utilizaremos terms aggregation cuando queremos extraer facetas de campos de tipo texto. En concreto, la agregación Terms Aggregation sólo funciona sobre campos de tipo **keyword**. Si el campo a nalalizar es de tipo text, podemos utilizar los multi-field para indexarlo también como tipo keyword.

Si queremos implementar la faceta "autores", es decir que elasticsearch nos indique cueantos autores diferentes existen en los documentos indexados para poder implementar el filtro de las facetas de nuestro buscador, realizaremos una gregación por el campo "author" que es de tipo keyword.

`
GET recipes/_search?size=0
{
    "aggs" : {
        "authors" : {
            "terms" : { "field" : "author" } 
        }
    }
}
`

En la información devuelta vamos a fijarnos en el campo "aggregations". ¿Podrídas contestar a las siguietnes preguntas? 

1. Para la faceta "authors" ¿cuántos buckets diferentes hay?
2. Cuantos documentos ha encontrado que contenga el nombre Staff en la faceta autor?
3. ¿Hay algún orden en el que devuelva los buckets?

Por defecto Elasticsearch devuelve los 10 buckets con más elementos. Si queremos cambiar este comportamiento, podemos utilizar el parámetro size:

`
GET recipes/_search?size=0
{
    "aggs" : {
        "authors" : {
            "terms" : {
                "field" : "author",
                "size" : 5
            }
        }
    }
}
`

1. ¿Cuántos buckets devuelve ahora esta sentencia?

Elasticsearch también nos permite cambiar el orden en el que devuelve los buckets. Por ejemplo, si queremos devolver los bockes ordenados de forma descendente por el número de documentos que contiene:

`
GET recipes/_search?size=0
{
    "aggs" : {
        "authors" : {
            "terms" : {
                "field" : "author",
                 "order" : { "_count" : "desc" }
            }
        }
    }
}
`

1. ¿Cómo ordena los buckts esta sentencia?

Pero además también podemos ordenar el resultado por el valor de la clave del bucket, por ejemplo, si queremos ordenar los resultados por orden alfabético:

`
GET recipes/_search?size=0
{
    "aggs" : {
        "authors" : {
            "terms" : {
                "field" : "author",
                "order" : { "_key" : "asc" }
            }
        }
    }
}
`

También podemos especificar el número mínimo de documentos que tiene que haber en cada bucket para que se incluya en el resultado.

`
GET recipes/_search?size=0
{
    "aggs" : {
        "authors" : {
            "terms" : {
                "field" : "author",
                "min_doc_count": 400
            }
        }
    }
}
`

1. ¿Cuántos buckets devuelve ahora esta sentencia?

## 2. Range Aggregation

Se utilizan sobre campos numéricos y permiten definir el rango de valores que puede tener cada bucket. Para ello se especifica el valor máximo y mínimo entre los que tien que estar el valor de una faceta para que se incluya en ese bucket.

`
GET recipes/_search?size=0
{
    "aggs" : {
        "ratings" : {
            "range" : {
                "field" : "rating.ratingValue",
                "ranges" : [
                    { "to" : 2.0 },
                    { "from" : 2.0, "to" : 4.0 },
                    { "from" : 4.0 }
                ]
            }
        }
    }
}
`

1. ¿Cuantos buckets nos devuelve esta consulta?

Podemos decirle a Elasticsearch que incluya como nombre del bucket el rango definido utilizando el parámetro "keyed" en la query:

`
GET recipes/_search?size=0
{
    "aggs" : {
        "ratings" : {
            "range" : {
                "field" : "rating.ratingValue",
                "keyed" : true,
                "ranges" : [
                    { "to" : 2.0 },
                    { "from" : 2.0, "to" : 4.0 },
                    { "from" : 4.0 }
                ]
            }
        }
    }
}
`

1. ¿Qué nombre tienen ahora los buckets?

Si queremos darle la misma semántica a los nombres que en el dominio que estamos modelando, podemos definir un nombre a cada bucket:

`
GET recipes/_search?size=0
{
    "aggs" : {
        "ratings" : {
            "range" : {
                "field" : "rating.ratingValue",
                "keyed" : true,
                "ranges" : [
                    { "key" : "bad", "to" : 2.0 },
                    { "key" : "good","from" : 2.0, "to" : 4.0 },
                    { "key" : "the best","from" : 4.0 }
                ]
            }
        }
    }
}
`

## 3. Date Range Aggregation

Funciona igual que la agregación Range que hemos visto antes, pero con campos de tipo fecha. Además permite expresiones específicas para fechas para definir los rangos.

Entra en este enlace para ver las expresiones matemáticas que se pueden utilizar: https://www.elastic.co/guide/en/elasticsearch/reference/current/common-options.html#date-math

`
POST recipes/_search?size=0
{
    "aggs": {
        "date": {
            "date_range": {
                "field": "date",
                "format": "MMMM yyyy",
                "ranges": [
                    { "to": "December 2020" },
                    { "from": "January 2021", "to": "December 2021"},
                    { "from": "January 2022" } 
                ]
            }
        }
    }
}
`

## 4. Histogram Aggregation

Se tuiliza con campos de tipo numérico. Permite realizar buckets en función de la sitribución de los datos de un campo. Para ello es necesario definir el número fijo de valores que puede haber entre el valor máximo y mínimo del bucket.

Si lo que queremos es tener un bucket por cada valor del rating, ejecutaríamos la siguiente consulta:

` 
POST recipes/_search?size=0
{
    "aggs" : {
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1
            }
        }
    }
}
`


Como pasara con el Terms Aggregation, se puede definir el número mínimo de documentos que tiene que tener el bucket para que aparezca en el resultado.

`
POST recipes/_search?size=0
{
    "aggs" : {
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1,
                "min_doc_count" :1000
            }
        }
    }
}
`

También permiten que el nombre del bucket sea el rango del intervalo.

`
POST recipes/_search?size=0
{
    "aggs" : {
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1,
                "keyed" : true
            }
        }
    }
}
`

## 5. Date Histogram Aggregation

Es igual que el Histogram Aggregation, pero con campos de tipo fecha. En este caso los inervalos pueden ser:

* minute (m, 1m)
* hour (h, 1h)
* day (d, 1d)
* week (w, 1w)
* month (M, 1M)
* quarter (q, 1q)
* year (y, 1y)

`
POST recipes/_search?size=0
{
    "aggs" : {
        "post_over_time" : {
            "date_histogram" : {
                "field": "date",
                "calendar_interval": "1y",
                "min_doc_count": 1
            }
        }
    }
}
`

# Construir el buscador

Para poder terminar de implementar nuestro buscador, además de poder mostrar los valores de las diferentes facetas, no interesa poder filtrar los resultados por los valores de las facetas seleccionados y a su vez poder regenerar de forma dinámica los filtros de las facetas teniendo en cuenta el nuevo resultado tras aplicar el filtro. 

El lenguaje de consulta de Elasticsearch permite añadir un contexto de agregación junto al contexto de búsqueda y filtrado. Con el resultado de la búsqueda se adjunta el resultado de la agregación.

La agregación siempre se aplica al subconjunto de documentos que cumplen los criterios de búsqueda definidos en el contexto de búsqueda y filtrado.

## Construir el filtro de facetas

El primer paso para constuir el buscador es consutruir la consulta que nos devuelva las facetas y sus valores. 
Vamos a poner en práctica lo que hemos aprendido sobre las bucket aggregations.

Queremos construir un filtro de facetas con las siguientes facetas:
* Autor
* Valoración
* Fecha de publicación

`
GET recipes/_search?size=0
{
    "aggs" : {
        "author" : {
            "terms" : {
                "field" : "author",
                "size" : 10,
                "order" : { "_key" : "asc" }
            }
        },
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1,
                "keyed" : true
            }
        },
        "date": {
          "date_histogram" : {
                "field": "date",
                "calendar_interval": "1y",
                "min_doc_count": 1,
                "order" : { "_key" : "desc" }
            }
        }
    }
}
`

Una vez que construimos el filtro de facetas, lo que esperamos es que el usuario lo utilice, por tanto puede seleccionar algún valor del filtro. Por lo tanto como resultado de la búsqueda tendremos que devolver los documentos filtrados por el valor o valores seleccionados, junto con las nuevos valores para las facetas.

Vamos a ver como sería la consulta. Imaginemos que el usuario ha seleccionado el valor 'Jennifer Iserloh' parala faceta "author":

`
GET recipes/_search
{
    "query": {
      "term": {
        "author": {
          "value": " Jennifer Iserloh",
          "case_insensitive": true
        }
      }    
    }, 
    "aggs" : {
        "author" : {
            "terms" : {
                "field" : "author",
                "size" : 10,
                "order" : { "_key" : "asc" }
            }
        },
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1,
                "keyed" : true
            }
        },
        "date": {
          "date_histogram" : {
                "field": "date",
                "calendar_interval": "1y",
                "min_doc_count": 1,
                "order" : { "_key" : "desc" }
            }
        }
    }
}
`

`
GET recipes/_search
{
    
    "query": {
        "bool": {
            "should": [
              { "term": { 
                "author": {
                  "value": " Jennifer Iserloh",
                  "case_insensitive": true
                }
              }},
              { "range" : {
                "rating.ratingValue": {
                    "gte": 3,
                    "lte": 4
                }
              }}
            ]
        }
    },
    "aggs" : {
        "author" : {
            "terms" : {
                "field" : "author",
                "size" : 10,
                "order" : { "_key" : "asc" }
            }
        },
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1,
                "keyed" : true
            }
        },
        "date": {
          "date_histogram" : {
                "field": "date",
                "calendar_interval": "1y",
                "min_doc_count": 1,
                "order" : { "_key" : "desc" }
            }
        }
    }
}
`

## Juntar la busqueda recomendada con la facetas

Podemos juntar la busqueda recomendada junto con el filtro de facetas. Igual que en el caso de los filtros, el calculo de los buckets de las facetas se realizará sobre el resultado de la búsqueda realizada para obtener las usgerencias.

`
GET recipes/_search
{
  "query": {
    "match": {
      "title.ngram": {
        "query": "ble"
      }
    }
  },
    "aggs" : {
        "author" : {
            "terms" : {
                "field" : "author",
                "size" : 10,
                "order" : { "_key" : "asc" }
            }
        },
        "rating" : {
            "histogram" : {
                "field" : "rating.ratingValue",
                "interval" : 1,
                "keyed" : true
            }
        },
        "date": {
          "date_histogram" : {
                "field": "date",
                "calendar_interval": "1y",
                "min_doc_count": 1,
                "order" : { "_key" : "desc" }
            }
        }
    }
}
`